In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_shishang_fengxiangbiao=pd.DataFrame(columns=columns)
channel_shishang_fengxiangbiao

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[11]/div/a/span').click()  #选择“时尚”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[11]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'时尚'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').click()  #选择“风尚标”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').text
sub_channel

'风尚标'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [ ]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_shishang_fengxiangbiao.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-23 00:27:25.957746


<ipython-input-9-3a1195bfd3c9>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-3a1195bfd3c9>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-9-3a1195bfd3c9>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1x7411i7au视频属性的抓取
BV号为1x7411i7au的视频属性抓取结束


<ipython-input-9-3a1195bfd3c9>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1pJ41177iW视频属性的抓取
BV号为1pJ41177iW的视频属性抓取结束
开始BV号为1gJ411E7HA视频属性的抓取
BV号为1gJ411E7HA的视频属性抓取结束
开始BV号为1E7411i74h视频属性的抓取
BV号为1E7411i74h的视频属性抓取结束
开始BV号为1QJ41177sR视频属性的抓取
BV号为1QJ41177sR的视频属性抓取结束
开始BV号为1m7411i7t7视频属性的抓取
BV号为1m7411i7t7的视频属性抓取结束
开始BV号为1GJ411G7ET视频属性的抓取
BV号为1GJ411G7ET的视频属性抓取结束
开始BV号为14J411H7k5视频属性的抓取
BV号为14J411H7k5的视频属性抓取结束
开始BV号为1R7411z7jJ视频属性的抓取
BV号为1R7411z7jJ的视频属性抓取结束
开始BV号为1rJ411j7B9视频属性的抓取
BV号为1rJ411j7B9的视频属性抓取结束
开始BV号为1PJ41157uK视频属性的抓取
BV号为1PJ41157uK的视频属性抓取结束
开始BV号为1Y7411v7dH视频属性的抓取
BV号为1Y7411v7dH的视频属性抓取结束
开始BV号为1H7411v7ZN视频属性的抓取
BV号为1H7411v7ZN的视频属性抓取结束
开始BV号为1kJ411V7ZN视频属性的抓取
BV号为1kJ411V7ZN的视频属性抓取结束
开始BV号为1TJ411p79m视频属性的抓取
BV号为1TJ411p79m的视频属性抓取结束
开始BV号为1e7411i7wh视频属性的抓取
BV号为1e7411i7wh的视频属性抓取结束
开始BV号为16J411G7nF视频属性的抓取
BV号为16J411G7nF的视频属性抓取结束
开始BV号为1k7411v79n视频属性的抓取
BV号为1k7411v79n的视频属性抓取结束
开始BV号为1w7411C7ja视频属性的抓取
BV号为1w7411C7ja的视频属性抓取结束
开始BV号为1NJ411j7S5视频属性的抓取
BV号为1NJ411j7S5的视频属性抓取结束
开始BV号为1T7411q7Fq视频属性的抓取
BV号为1T7411q7Fq的视频属性抓取结束
开始BV号为1HJ41177kx视频属性的抓取
BV号为1HJ41177kx的视

In [15]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-15-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [10]:
channel_shishang_fengxiangbiao.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [11]:
channel_shishang_fengxiangbiao.to_json('channel_shishang_fengxiangbiao.json')

In [23]:
pd.set_option('display.max_rows', None)

In [16]:
channel_shishang_fengxiangbiao

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1x7411i7au,1月,硬核的半佛仙人,个人认证,508.8万,131,【半佛】买不起奢侈品就是穷人吗？不，是聪明人,时尚,风尚标,普通视频,自制,"[奢侈品, 服饰, 时尚, 风尚标, 学习, 消费, 防骗, 寒假Vlog-一起学习吧, 品...",17:17,749.0万,63.4万,66.3万,38.2万,32.0万,24470,4.9万,否
1pJ41177iW,1月,花小雕,个人认证,32.7万,62,如果时尚博主说真话，都市丽人秘辛初公开,时尚,风尚标,普通视频,自制,"[搞笑, 时尚, 风尚标, 恶搞, 时尚, 高能, 娱乐, 沙雕]",01:50,46.6万,2.5万,3550,1679,1400,1008,486,否
1gJ411E7HA,1月,IC实验室,个人认证,110.2万,103,【消费战争01】联合利华vs宝洁，中国洗发水七十年厮杀混战（上）-IC实验室出品,时尚,风尚标,活动作品,自制,"[秀场, 服饰, 时尚, 风尚标, 宝洁, 我的职业故事, 时尚, 品牌, 市场, 洗发水,...",13:37,36.2万,1.7万,6666,4308,2091,1074,3493,否
1E7411i74h,1月,阿菊Addy,个人认证,11.2万,116,外国人如何应对尴尬时刻？分享妙招！！跳出“尴尬区”【阿菊】,时尚,风尚标,活动作品,自制,"[秀场, 搞笑, 时尚, 风尚标, 时尚星计划, 时尚, 技巧, 日常, 干货, 尴尬场面,...",09:05,35.6万,2.8万,1.1万,2.0万,683,1313,2127,否
1QJ41177sR,1月,SeeC茜茜才不困,个人认证,24.5万,45,花20分钟在首尔打HPV疫苗？靠谱吗？公立医院预约注射全攻略,时尚,风尚标,活动作品,自制,"[种草, 时尚, 风尚标, 时尚星计划, 时尚, 日常, 生活, VLOG, 首尔, 分享,...",03:35,32.6万,3373,737,1703,639,720,179,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1Ty4y1r7mW,10月,桐生二狗,无,4.3万,1712,「街拍」2019日本东京街头 | HOHOJOO,时尚,风尚标,普通视频,转载,"[潮流, 日本, 服饰, 时尚, 风尚标, 街头, 东京, 搭配, HOHOJOO김크리, ...",01:58,4.0万,953,141,778,180,54,55,否
1fK4y177Xh,10月,Snow雪小雪_,无,9万,57,百万衣橱大公开之CHANEL香奈儿包包｜最便宜的居然只要100块钱！,时尚,风尚标,活动作品,自制,"[奢侈品, 秀场, 服饰, 时尚, 风尚标, 发现新世界, 包包, 时尚, 香奈儿, FAS...",13:17,4.0万,1443,259,444,139,55,217,否
1Hy4y1B7L6,10月,偶尔有点小情绪咯,无,2323,5424,【李佳琦试用】金靖在李佳琦面前，简直就是行走的“段子手”，李佳琦：求放过,时尚,风尚标,普通视频,转载,"[李佳琦, 搞笑, 时尚, 风尚标, 明星, 直播, 时尚, 爆笑, 金靖, 淘宝直播, f...",00:31,4.0万,1888,0,72,5,4,0,否
1Xr4y1c7Cf,10月,WBJewelry,无,8840,72,Knowknow卡地亚手表VVS级钻石镶嵌,时尚,风尚标,普通视频,自制,"[手表, 奢侈品, 嘻哈, 时尚, 风尚标, WBJ定制珠宝, MEDM, 10月打卡挑战W...",03:49,3.9万,564,35,102,87,70,33,否
